# Split FastMRI

In [1]:
# %cd ../
import os
import re
import numpy as np
import h5py
from tqdm import tqdm
from os.path import join
from utils.kspace import spatial2kspace, kspace2spatial

from config import PATH

In [2]:
def getFieldStrength(h_file):
    ismhd = h_file['ismrmrd_header'][()]
    match = re.findall(r'<systemFieldStrength_T>\d\.\d*</systemFieldStrength_T>', str(ismhd))[0]
    match = match.replace('<systemFieldStrength_T>', '')
    match = match.replace('</systemFieldStrength_T>', '')
    return 1.5 if float(match) < 2.0 else 3.0

def getAcquisitionType(h_file):
    return 'PD' if h_file.attrs['acquisition'] == 'CORPD_FBK' else 'PDFS'

In [4]:
DIR_PATH = '/home/a_razumov/smbmount_a_razumov/fastMRIdatasets/singlecoil_val/'
hf_map = {}

for fname in tqdm(os.listdir(DIR_PATH)):
    with h5py.File(os.path.join(DIR_PATH, fname)) as hf:
        hf_map[fname] = (getFieldStrength(hf), getAcquisitionType(hf))
        
PD_1T = [f for f,v in hf_map.items() if v == (1.5, 'PD')]
PD_3T = [f for f,v in hf_map.items() if v == (3.0, 'PD')]
PDFS_1T = [f for f,v in hf_map.items() if v == (1.5, 'PDFS')]
PDFS_3T = [f for f,v in hf_map.items() if v == (3.0, 'PDFS')]

  0%|          | 0/199 [00:00<?, ?it/s]<ipython-input-4-3760a111179a>:5: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(os.path.join(DIR_PATH, fname)) as hf:
100%|██████████| 199/199 [00:00<00:00, 282.98it/s]


In [ ]:
for f_list, name in [(PD_1T, 'PD_1T'), (PD_3T, 'PD_3T'), (PDFS_1T, 'PDFS_1T'), (PDFS_3T, 'PDFS_3T')]:
    with h5py.File(PATH.VAL_PATH + '_%s.h5' % name, 'w') as f:
        for fname in tqdm(f_list):
            hf = h5py.File(join(DIR_PATH, fname))
            f.create_dataset(fname, data=hf['kspace'][:])
            hf.close()

In [ ]:
DIR_PATH = '/home/a_razumov/smbmount_a_razumov/fastMRIdatasets/singlecoil_train/'
hf_map = {}

for fname in tqdm(os.listdir(DIR_PATH)):
    with h5py.File(os.path.join(DIR_PATH, fname)) as hf:
        hf_map[fname] = (getFieldStrength(hf), getAcquisitionType(hf))
        
PD_1T = [f for f,v in hf_map.items() if v == (1.5, 'PD')]
PD_3T = [f for f,v in hf_map.items() if v == (3.0, 'PD')]
PDFS_1T = [f for f,v in hf_map.items() if v == (1.5, 'PDFS')]
PDFS_3T = [f for f,v in hf_map.items() if v == (3.0, 'PDFS')]

set(PD_1T + PD_3T + PDFS_1T + PDFS_3T) == set(hf_map.keys())

In [ ]:
for f_list, name in [(PD_1T, 'PD_1T'), (PD_3T, 'PD_3T'), (PDFS_1T, 'PDFS_1T'), (PDFS_3T, 'PDFS_3T')]:
    with h5py.File(PATH.TRAIN_PATH + '_%s.h5' % name, 'w') as f:
        for fname in tqdm(f_list):
            hf = h5py.File(join(DIR_PATH, fname))
            f.create_dataset(fname, data=hf['kspace'][:])
            hf.close()

# Create Corrupted Datasets

In [2]:
%env CUDA_VISIBLE_DEVICES=2
import os
from os.path import join

import math
import random
import numpy as np
import pylab as plt
import torch
import torch.nn.functional as F
import h5py

from utils.utils import plot_motion_vector
from utils.fastmri import FastMRITransform, DemotionFastMRIh5Dataset
from utils.kspace import RandomMotionTransform
from utils.unet import Unet

from tqdm import tqdm
import skimage.data
from utils.utils import l1_loss, t2i, psnr, ssim

import sys
sys.path.append(PATH.NUFFT_PATH)
import nufft
from torch.fft import fftshift, ifftshift, fftn, ifftn

Ft = lambda x : fftshift(fftn(ifftshift(x, dim=(-1, -2)), dim=(-1, -2)), dim=(-1, -2))
IFt = lambda x : ifftshift(ifftn(fftshift(x, dim=(-1, -2)), dim=(-1, -2)), dim=(-1, -2))
from IPython.display import clear_output

random.seed(228)
torch.manual_seed(228)
torch.cuda.manual_seed(228)
np.random.seed(228)

env: CUDA_VISIBLE_DEVICES=2


In [3]:
# Import motion class for the creation of corruptions
DIR_PATH = '/home/a_razumov/small_datasets/small_fastMRIh5_PD_3T/val_small_PD_3T.h5'

dataset = DemotionFastMRIh5Dataset(
    DIR_PATH,
    None,
    RandomMotionTransform(xy_max=5, theta_max=1.5, num_motions=5,
                          center_fractions=0.08, wave_num=6,
                          motion_type='randomize_harmonic',
                          noise_lvl=0), z_slices=0.1)

/home/ekuzmina/AUTOFOCUSING-PLUS/utils/fastmri.py:101: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.hf = h5py.File(hf_path)


In [4]:
with h5py.File(PATH.VAL_PATH + PATH.VAL_NAME, 'w') as f:
    
    for fname in tqdm(range(0, len(dataset), 2)):
        batch = dataset[fname]
        gt_ks = batch['target_k_space']
        gt_ks = gt_ks[0] + 1j * gt_ks[1]
        ks = batch['k_space']
        ks = ks[0] + 1j * ks[1]
        
        ksp_gt_ksp = np.concatenate((ks[None].numpy(), gt_ks[None].numpy()), axis=0)
            
        f.create_dataset(str(fname), data=ksp_gt_ksp)

100%|██████████| 53/53 [00:26<00:00,  2.03it/s]


## Validate Metrics of Created Dataset

In [ ]:
hf = h5py.File(VAL_PATH + VAL_NAME)

metrics = []
for f in tqdm(sorted(list(hf.keys()))):
    t = hf[f]
    ks = t[0]
    gt_ks = t[1]
    
    metrics.append(calc_metrics(torch.from_numpy(kspace2spatial(ks))[None, None],
                                torch.from_numpy(kspace2spatial(gt_ks))[None, None]))
    plt.imshow(kspace2spatial(ks))
    plt.show()
    
val_psnr = [d['psnr'] for d in metrics]
val_ssim = [[d['ssim'] for d in metrics]]
val_ms = [d['ms_ssim'] for d in metrics]
val_vif = [[d['vif_p'] for d in metrics]]
val_psnr = np.array(val_psnr)
val_ssim = np.array(val_ssim)
val_ms = np.array(val_ms)
val_vif = np.array(val_vif)
print('PSNR: ', val_psnr.mean(), ' + ', val_psnr.std())
print('SSIM: ', val_ssim.mean(), ' + ', val_ssim.std())
print('MS-SSIM: ', val_ms.mean(), ' + ', val_ms.std())
print('VIF: ', val_vif.mean(), ' + ', val_vif.std())